# Tutorial 3-3: Trust but Verify – "Gradient Checking"

**Course:** CSEN 342: Deep Learning  
**Topic:** Numerical Differentiation, Gradient Checks, and Debugging Backprop

## Objective
Implementing backpropagation from scratch is prone to subtle bugs (e.g., off-by-one errors, sign flips). Even if your model trains, the gradients might be slightly wrong, leading to poor convergence.

In this tutorial, first we will learn how to **verify** our gradients using **Finite Difference Approximation**. We will:
1.  **Implement Numerical Gradients:** Write a function to compute the "Two-sided derivative" approximation: $\frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon}$.
2.  **The Bug Hunt:** You will be given a broken manual backprop implementation. You must use your gradient checker to identify which layer has the bug.
3.  **Fix and Verify:** Correct the math and confirm that the difference between analytic and numerical gradients drops below $10^{-7}$.

However, in the real world you will not be writing python/numpy code for training your network. You will use Pytorch for that. While PyTorch provides a vast library of standard operations, advanced research often requires implementing **custom activation functions**, loss functions, or layers that don't exist yet.

To do this, you must tell PyTorch exactly how to calculate the gradient (the `backward` pass). If your math is even slightly off, the model might train but converge to a suboptimal solution. We will:

4.  **Build a Custom Op:** Implement a custom **Sigmoid** activation function by subclassing `torch.autograd.Function`.
5.  **Introduce a Bug:** We will intentionally make a mistake in the derivative calculation.
6.  **Verify with `gradcheck`:** Use PyTorch's built-in utility to mathematically verify our implementation against a finite-difference approximation.

---

## Part 1: Numerical Differentiation

Calculus tells us that the derivative is the slope of the tangent line. We can approximate this slope by evaluating the function at two very close points ($x+\epsilon$ and $x-\epsilon$).

$$ \frac{df}{dx} \approx \frac{f(x+\epsilon) - f(x-\epsilon)}{2\epsilon} $$

This is slow (requires two forward passes for *every* parameter), but it is the "ground truth" we use to check our fast backprop code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
    """ 
    a naive implementation of numerical gradient of f at x 
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """
    fx = f(x) # evaluate function value at original point
    grad = np.zeros_like(x)
    
    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        oldval = x[ix]
        
        # Calculate f(x + h)
        x[ix] = oldval + h
        fxph = f(x)
        
        # Calculate f(x - h)
        x[ix] = oldval - h
        fxmh = f(x)
        
        # Restore value
        x[ix] = oldval 
        
        # Compute the partial derivative with centered formula
        grad[ix] = (fxph - fxmh) / (2 * h) 
        
        if verbose:
            print(ix, grad[ix])
        it.iternext() 

    return grad

print("Numerical Gradient Checker Defined.")

Numerical Gradient Checker Defined.


### 1.1 Simple Test
Let's verify our checker on a simple function: $f(x) = x^2$. The derivative should be $2x$. 
At $x=5$, we expect the gradient to be $10$.

In [2]:
def square_func(x):
    return np.sum(x**2)

x = np.array([5.0])
grad = eval_numerical_gradient(square_func, x)
print(f"Gradient at x=5: {grad[0]} (Expected: 10.0)")

(0,) 9.999999999621423
Gradient at x=5: 9.999999999621423 (Expected: 10.0)


---

## Part 2: The Bug Hunt

Below is a class `BrokenTwoLayerNet` that implements a simple neural network. However, **one of the backpropagation lines has a math error**.

Your job is to use `eval_numerical_gradient` to find which parameter (W1, b1, W2, or b2) has the incorrect gradient.

In [3]:
class BrokenTwoLayerNet(object):
    def __init__(self, input_size, hidden_size, output_size, std=1e-4):
        self.params = {}
        self.params['W1'] = std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def loss(self, X, y=None):
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        N, D = X.shape

        # --- Forward Pass ---
        # Layer 1
        h1_score = X.dot(W1) + b1
        h1 = np.maximum(0, h1_score) # ReLU
        
        # Layer 2
        scores = h1.dot(W2) + b2
        
        if y is None:
            return scores

        # Softmax Loss
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        correct_logprobs = -np.log(probs[range(N), y])
        loss = np.sum(correct_logprobs) / N
        
        # --- Backward Pass ---
        grads = {}
        
        # dScores
        dscores = probs
        dscores[range(N), y] -= 1
        dscores /= N

        # Backprop W2, b2
        grads['W2'] = h1.T.dot(dscores)
        grads['b2'] = np.sum(dscores, axis=0)

        # Backprop Hidden Layer
        dhidden = dscores.dot(W2.T)
        dhidden[h1 <= 0] = 0

        # Backprop W1, b1
        # BUG IS LIKELY HERE
        grads['W1'] = X.T.dot(dhidden) * 2.0  # <--- Suspicious multiplication?
        grads['b1'] = np.sum(dhidden, axis=0)

        return loss, grads

print("Broken Model Defined.")

Broken Model Defined.


### 2.1 Running the Check

We generate some dummy data and run the check. The `rel_error` function calculates the relative difference between two matrices. 

$$ \text{rel\_error} = \frac{|x - y|}{\max(|x| + |y|, 1e-8)} $$

In [4]:
def rel_error(x, y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Create tiny data
input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
    np.random.seed(0)
    return BrokenTwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

net = init_toy_model()
X, y = init_toy_data()

# 1. Analytic Gradient (from our backprop code)
loss, grads = net.loss(X, y)

# 2. Numerical Gradient (Ground Truth)
# We check each parameter separately
for param_name in grads:
    f = lambda w: net.loss(X, y)[0] # Wrapper to compute loss for current weights
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    
    error = rel_error(param_grad_num, grads[param_name])
    print(f'{param_name} max relative error: {error:.2e}')

W2 max relative error: 6.37e-10
b2 max relative error: 4.45e-11
W1 max relative error: 3.33e-01
b1 max relative error: 2.74e-09


**Discussion:** 
You should see that `W2`, `b2`, and `b1` have very small errors (around $10^{-8}$ or less). 
However, `W1` has a huge error (likely between $0.1$ and $1.0$). This confirms the bug is in the `W1` calculation.

**The Fix:** Look closely at the `grads['W1']` line in `BrokenTwoLayerNet`. Remove the incorrect `* 2.0` multiplier.

---

## Part 3: The Fixed Model

Correct the code below and verify that all gradients pass the check.

In [5]:
# Copy of the class with the fix applied
class FixedTwoLayerNet(BrokenTwoLayerNet):
    def loss(self, X, y=None):
        # ... (Forward pass is same) ...
        W1, b1 = self.params['W1'], self.params['b1']
        W2, b2 = self.params['W2'], self.params['b2']
        N, D = X.shape

        # Forward
        h1_score = X.dot(W1) + b1
        h1 = np.maximum(0, h1_score)
        scores = h1.dot(W2) + b2
        
        if y is None: return scores
        
        # Loss
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        correct_logprobs = -np.log(probs[range(N), y])
        loss = np.sum(correct_logprobs) / N

        # Backward
        grads = {}
        dscores = probs
        dscores[range(N), y] -= 1
        dscores /= N

        grads['W2'] = h1.T.dot(dscores)
        grads['b2'] = np.sum(dscores, axis=0)

        dhidden = dscores.dot(W2.T)
        dhidden[h1 <= 0] = 0

        # FIXED LINE:
        grads['W1'] = X.T.dot(dhidden) # Removed * 2.0
        grads['b1'] = np.sum(dhidden, axis=0)

        return loss, grads

# Run Check Again
print("Checking Fixed Model:")
net_fixed = FixedTwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)
net_fixed.params = net.params # Copy weights from broken net to keep test consistent

loss, grads_fixed = net_fixed.loss(X, y)

for param_name in grads_fixed:
    f = lambda w: net_fixed.loss(X, y)[0]
    param_grad_num = eval_numerical_gradient(f, net_fixed.params[param_name], verbose=False)
    error = rel_error(param_grad_num, grads_fixed[param_name])
    print(f'{param_name} max relative error: {error:.2e}')

Checking Fixed Model:
W2 max relative error: 6.37e-10
b2 max relative error: 4.45e-11
W1 max relative error: 1.53e-09
b1 max relative error: 2.74e-09


## Part 4: Anatomy of a Custom Autograd Function

To create a new operation in PyTorch, you must define a class that inherits from `torch.autograd.Function` and implements two static methods:

1.  **`forward(ctx, input)`**: Computes the output. You also use `ctx.save_for_backward(input)` to cache data needed for the gradient later.
2.  **`backward(ctx, grad_output)`**: Computes the gradient of the loss with respect to the input. It receives the `grad_output` (upstream gradient) and must return `grad_input` (local gradient * upstream gradient).

### The Math: Sigmoid
$$ f(x) = \sigma(x) = \frac{1}{1 + e^{-x}} $$

**Derivative:**
$$ \frac{\partial f}{\partial x} = \sigma(x) \cdot (1 - \sigma(x)) $$

In [6]:
import torch
from torch.autograd import Function

class BuggySigmoid(Function):
    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation.
        """
        output = 1.0 / (1.0 + torch.exp(-input))
        
        # Save the output for the backward pass
        # (Recall: sigmoid derivative is easier to calculate using the output)
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        output, = ctx.saved_tensors
        
        # --- THE BUG IS HERE ---
        # Correct Math: output * (1 - output)
        # Our Math:     output * (1 + output)  <-- Oops!
        grad_sigmoid = output * (1.0 + output)
        
        # Chain Rule: local_grad * upstream_grad
        return grad_sigmoid * grad_output

# Wrap it in an alias so we can apply it like a function
buggy_sigmoid = BuggySigmoid.apply

print("BuggySigmoid defined.")

BuggySigmoid defined.


---

## Part 5: The Check (Why validation matters)

If we tried to train a neural network with this bug, it might actually "work" (the loss might decrease), but it would behave strangely or converge slowly. Visual inspection of code is hard. Numerical inspection is better.

PyTorch provides `torch.autograd.gradcheck`. It:
1.  Computes the analytical gradient using your `backward` method.
2.  Computes the numerical gradient using finite differences ($f(x+\epsilon) - f(x-\epsilon)$).
3.  Compares them and raises an error if they differ.

**Note:** `gradcheck` requires inputs to be **double precision** (`float64`) for numerical stability.

In [7]:
from torch.autograd import gradcheck

# 1. Create input data
# We must enable requires_grad=True so PyTorch tracks it
# We use double precision (float64) to minimize floating point errors during the check
input_test = torch.randn(5, requires_grad=True, dtype=torch.float64)

print("Running gradcheck on BuggySigmoid...")

try:
    # gradcheck takes (function, inputs, eps=...)
    # If the check passes, it returns True. If it fails, it raises a RuntimeError.
    test = gradcheck(buggy_sigmoid, input_test, eps=1e-6, atol=1e-4)
    print("Test Passed!")
except RuntimeError as e:
    print("\nTest FAILED! PyTorch caught the bug.")
    print("Error details:")
    print(e)

Running gradcheck on BuggySigmoid...

Test FAILED! PyTorch caught the bug.
Error details:
Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[0.1957, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1723, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1957, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.1337, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.2496]], dtype=torch.float64)
analytical:tensor([[1.2702, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2703, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2702, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.5484, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7085]], dtype=torch.float64)



### Discussion
You should see a failure message. PyTorch compares the **Numerical** Jacobian (calculated by perturbing inputs) with the **Analytical** Jacobian (calculated by your buggy code). Because we used `(1 + output)` instead of `(1 - output)`, the numbers will be wildly different.

---

## Part 6: The Fix

Now, implement the class correctly.

In [8]:
class CorrectSigmoid(Function):
    @staticmethod
    def forward(ctx, input):
        output = 1.0 / (1.0 + torch.exp(-input))
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        output, = ctx.saved_tensors
        
        # --- FIXED LOGIC ---
        grad_sigmoid = output * (1.0 - output)
        
        return grad_sigmoid * grad_output

correct_sigmoid = CorrectSigmoid.apply

print("Running gradcheck on CorrectSigmoid...")
try:
    test = gradcheck(correct_sigmoid, input_test, eps=1e-6, atol=1e-4)
    print(f"Test Passed? {test}")
except RuntimeError as e:
    print("Test FAILED!")
    print(e)

Running gradcheck on CorrectSigmoid...
Test Passed? True


Once verified, you can use `CorrectSigmoid.apply` just like any other PyTorch function inside a neural network.

In [ ]:
import torch.nn as nn

# Create a custom module wrapper
class MySigmoidLayer(nn.Module):
    def forward(self, x):
        return CorrectSigmoid.apply(x)

# Define a tiny model using our custom activation
model = nn.Sequential(
    nn.Linear(10, 5),
    MySigmoidLayer(),  # Using our custom autograd function
    nn.Linear(5, 1)
)

print(model)

# Verify we can backprop through it
dummy_input = torch.randn(2, 10)
output = model(dummy_input)
loss = output.mean()
loss.backward()

print("\nBackward pass successful. Gradients computed for first layer:")
print(model[0].weight.grad[0][:5]) # Print a few gradients

### Conclusion
Gradient Checking is a powerful tool. When you are writing custom CUDA kernels or new layer types (where you can't rely on PyTorch `autograd`), this is the primary way to verify your derivative calculus is correct. Similarly, you need to check your gradients when you  **extend PyTorch** by writing custom `Function` subclasses. Whenever you implement complex math operations not supported by the standard library, **always** run a gradient check before training!

**Note:** Numerical gradient checking is very slow. You should only use it for debugging on small networks, never for actual training.